# Instalação das bibliotecas utilizadas

Antes de iniciarmos a análise dos dados de Consumo no Google Colab, é essencial garantir que todas as bibliotecas necessárias estejam devidamente instaladas e atualizadas. No ambiente do Colab, embora muitas bibliotecas já venham pré-instaladas, pode ser necessário instalar ou atualizar pacotes adicionais para garantir que o código funcione corretamente, como por exemplo o MeteoStat, biblioteca utilizada para coletar os dados climáticos.

In [ ]:
!pip install meteostat

In [ ]:
# Importação das bibliotecas

import numpy as np
import pandas as pd
import gdown
import plotly.graph_objects as go
from meteostat import Point, Daily
from datetime import datetime

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "consumo_2020": "1-GqmDmxn-DpXJnoU05mcZ3U7Dvx8M_ZG",
}

dataframes = {}

for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)
    df = pd.read_csv(arquivo_destino, sep=";")
    dataframes[key] = df


In [ ]:
df_19 = pd.read_csv("/content/dataset_consumo_2019.csv", delimiter=";")
df_20 = pd.read_csv("/content/dataset_consumo_2020.csv", delimiter=";")

# Estruturação dos datasets - 2019

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2019 na integra, com todas as colunas disponibilizadas pelo parceiro, e os dados climáticos, que serão utilizados.

**Desta forma, se precisarmos de mais colunas e features, será de fácil recuperação**

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define o intervalo de datas
start = datetime(2019, 1, 1)
end = datetime(2019, 12, 31)

# Busca os dados diários de clima para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados climáticos
df_monthly_weather_19 = data.resample('M').mean()

# Renomeia as colunas do DataFrame para termos mais descritivos em português
df_weather_19 = df_monthly_weather_19.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_19

A seguir iremos remover as colunas que julgamos não serem importantes para entender a mudança do consumo da água.

Aqui fizemos uma descrição estatística básica nas colunas e seus valores

In [ ]:
print("\nDescrição estatística básica:")
print(df_19.describe())

In [ ]:
# Remove as colunas desnecessárias do DataFrame
df_weather_19 = df_weather_19.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])
df_weather_19

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime, garantindo que as datas sejam reconhecidas corretamente
df_19['REFERENCIA'] = pd.to_datetime(df_19['REFERENCIA'])

# Cria uma nova coluna 'MES' extraindo o número do mês da coluna 'REFERENCIA'
df_19['MES'] = df_19['REFERENCIA'].dt.month

# Extrai o número do mês do índice (assumindo que o índice do DataFrame é uma coluna datetime) e cria uma nova coluna 'MES'
df_weather_19['MES'] = df_weather_19.index.month

# Agrupa os dados por mês e calcula a média do consumo medido ('CONS_MEDIDO') para cada mês
consumo_medio_19 = df_19.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina os dados de consumo médio com os dados meteorológicos com base na coluna 'MES'
df_combined_19 = pd.merge(consumo_medio_19, df_weather_19, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_19'
linhas, colunas = df_19.shape

print(f"O DataFrame de 2019 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Extrai os valores únicos da coluna 'MATRICULA' no DataFrame 'df_19'
matriculas_unicas = df_19['MATRICULA'].unique()

# Calcula o número total de valores únicos na coluna 'MATRICULA'
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2019 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total dos valores na coluna 'CONS_MEDIDO' no DataFrame 'df_19'
soma_consumo = df_19['CONS_MEDIDO'].sum()

print(f"Em 2019 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona o consumo médio mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_19['MES'],
    y=df_combined_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a temperatura média mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_19['MES'],
    y=df_combined_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlation_19 = df_combined_19['CONS_MEDIDO'].corr(df_combined_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_19:.2f}")

In [ ]:
# Obtém os valores únicos da coluna 'CATEGORIA' no DataFrame 'df_19'
categorias_unicas = df_19['CATEGORIA'].unique()

categorias_unicas

Este gráfico apresenta a análise do consumo médio de água na categoria **Pública** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_19 = df_19[df_19['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_19 = consumo_publica_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_19 = pd.merge(consumo_medio_publica_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_19['MES'],
    y=df_combined_publica_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_19['MES'],
    y=df_combined_publica_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Pública'
correlacao_pearson = df_combined_publica_19['CONS_MEDIDO'].corr(df_combined_publica_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.2f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Comercial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_19 = df_19[df_19['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_19 = consumo_comercial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_19 = pd.merge(consumo_medio_comercial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_19['MES'],
    y=df_combined_comercial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_19['MES'],
    y=df_combined_comercial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Comercial'
correlacao_pearson = df_combined_comercial_19['CONS_MEDIDO'].corr(df_combined_comercial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Residencial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_19 = df_19[df_19['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_19 = consumo_residencial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_19 = pd.merge(consumo_medio_residencial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_19['MES'],
    y=df_combined_residencial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_19['MES'],
    y=df_combined_residencial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Residencial'
correlacao_pearson = df_combined_residencial_19['CONS_MEDIDO'].corr(df_combined_residencial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Industrial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_19 = df_19[df_19['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_19 = consumo_industrial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_19 = pd.merge(consumo_medio_industrial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_19['MES'],
    y=df_combined_industrial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_19['MES'],
    y=df_combined_industrial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Industrial'
correlacao_pearson = df_combined_industrial_19['CONS_MEDIDO'].corr(df_combined_industrial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

Nesta seção, apresentamos uma análise detalhada do consumo médio mensal de água para diferentes categorias: Pública, Comercial, Residencial e Industrial. Ao comparar essas categorias lado a lado, podemos identificar padrões de consumo específicos, sazonalidades e variações que refletem as diferentes necessidades e comportamentos de cada grupo ao longo do ano.


In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_19[df_19['CATEGORIA'] == categoria]
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação

Nesta seção, exploramos o consumo médio mensal de água segmentado por tipo de ligação: Hidrometrado e Consumo Fixo. Essa análise permite observar como o consumo varia ao longo do ano para cada tipo de ligação.


In [ ]:
# Obtém os valores únicos da coluna 'TIPO_LIGACAO' no DataFrame 'df_19'
tipo_ligacao = df_19['TIPO_LIGACAO'].unique()

# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_19[df_19['TIPO_LIGACAO'] == tipo]
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()


### Consumo Médio Mensal por Subcategoria

Nesta seção, analisamos o consumo médio mensal de água segmentado por subcategoria. As subcategorias incluem Pública, Comercial, Residencial, Residencial Social, Industrial e Residencial Indubrasil.


In [ ]:
# Obtém os valores únicos da coluna 'SUB_CATEGORIA' no DataFrame 'df_19'
sub_categoria = df_19['SUB_CATEGORIA'].unique()


# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_19[df_19['SUB_CATEGORIA'] == sub_categoria]
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

# Estruturação dos datasets - 2020

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2020 na integra, com todas as colunas disponibilizadas pelo parceiro, e os dados climáticos, que serão utilizados.

**Desta forma, se precisarmos de mais colunas e features, será de fácil recuperação**

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define o intervalo de datas
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31)

# Busca os dados diários de clima para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados climáticos
df_monthly_weather_20 = data.resample('M').mean()

# Renomeia as colunas do DataFrame para termos mais descritivos em português
df_weather_20 = df_monthly_weather_20.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_20

A seguir iremos remover as colunas que julgamos não serem importantes para entender a mudança do consumo da água.

Aqui fizemos uma descrição estatística básica nas colunas e seus valores

In [ ]:
print("\nDescrição estatística básica:")
print(df_20.describe())

In [ ]:
# Remove as colunas desnecessárias do DataFrame
df_weather_20 = df_weather_20.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])
df_weather_20

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime, garantindo que as datas sejam reconhecidas corretamente
df_20['REFERENCIA'] = pd.to_datetime(df_20['REFERENCIA'])

# Cria uma nova coluna 'MES' extraindo o número do mês da coluna 'REFERENCIA'
df_20['MES'] = df_20['REFERENCIA'].dt.month

# Extrai o número do mês do índice (assumindo que o índice do DataFrame é uma coluna datetime) e cria uma nova coluna 'MES'
df_weather_20['MES'] = df_weather_20.index.month

# Agrupa os dados por mês e calcula a média do consumo medido ('CONS_MEDIDO') para cada mês
consumo_medio_20 = df_20.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina os dados de consumo médio com os dados meteorológicos com base na coluna 'MES'
df_combined_20 = pd.merge(consumo_medio_20, df_weather_20, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_20'
linhas, colunas = df_20.shape

print(f"O DataFrame de 2020 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Extrai os valores únicos da coluna 'MATRICULA' no DataFrame 'df_20'
matriculas_unicas = df_20['MATRICULA'].unique()

# Calcula o número total de valores únicos na coluna 'MATRICULA'
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2020 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total dos valores na coluna 'CONS_MEDIDO' no DataFrame 'df_20'
soma_consumo = df_20['CONS_MEDIDO'].sum()

print(f"Em 2020 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona o consumo médio mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_20['MES'],
    y=df_combined_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a temperatura média mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_20['MES'],
    y=df_combined_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlation_20 = df_combined_20['CONS_MEDIDO'].corr(df_combined_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_20:.2f}")

In [ ]:
# Obtém os valores únicos da coluna 'CATEGORIA' no DataFrame 'df_20'
categorias_unicas = df_20['CATEGORIA'].unique()

categorias_unicas

Este gráfico apresenta a análise do consumo médio de água na categoria **Pública** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_20 = df_20[df_20['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_20 = consumo_publica_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_20 = pd.merge(consumo_medio_publica_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_20['MES'],
    y=df_combined_publica_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_20['MES'],
    y=df_combined_publica_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Pública'
correlacao_pearson = df_combined_publica_20['CONS_MEDIDO'].corr(df_combined_publica_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.2f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Comercial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_20 = df_20[df_20['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_20 = consumo_comercial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_20 = pd.merge(consumo_medio_comercial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Comercial'
correlacao_pearson = df_combined_comercial_20['CONS_MEDIDO'].corr(df_combined_comercial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Residencial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_20 = df_20[df_20['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_20 = consumo_residencial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_20 = pd.merge(consumo_medio_residencial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_20['MES'],
    y=df_combined_residencial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_20['MES'],
    y=df_combined_residencial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Residencial'
correlacao_pearson = df_combined_residencial_20['CONS_MEDIDO'].corr(df_combined_residencial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Industrial** em relação à temperatura média ao longo dos meses.

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_20 = df_20[df_20['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_20 = consumo_industrial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_20 = pd.merge(consumo_medio_industrial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_20['MES'],
    y=df_combined_industrial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_20['MES'],
    y=df_combined_industrial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Industrial'
correlacao_pearson = df_combined_industrial_20['CONS_MEDIDO'].corr(df_combined_industrial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

Nesta seção, apresentamos uma análise detalhada do consumo médio mensal de água para diferentes categorias: Pública, Comercial, Residencial e Industrial. Ao comparar essas categorias lado a lado, podemos identificar padrões de consumo específicos, sazonalidades e variações que refletem as diferentes necessidades e comportamentos de cada grupo ao longo do ano de 2024.


In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_20[df_20['CATEGORIA'] == categoria]
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação

Nesta seção, exploramos o consumo médio mensal de água segmentado por tipo de ligação: Hidrometrado e Consumo Fixo. Essa análise permite observar como o consumo varia ao longo do ano para cada tipo de ligação.


In [ ]:
# Obtém os valores únicos da coluna 'TIPO_LIGACAO' no DataFrame 'df_20'
tipo_ligacao = df_20['TIPO_LIGACAO'].unique()

# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_20[df_20['TIPO_LIGACAO'] == tipo]
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()


### Consumo Médio Mensal por Subcategoria

Nesta seção, analisamos o consumo médio mensal de água segmentado por subcategoria. As subcategorias incluem Pública, Comercial, Residencial, Residencial Social, Industrial e Residencial Indubrasil.


In [ ]:
# Obtém os valores únicos da coluna 'SUB_CATEGORIA' no DataFrame 'df_20'
sub_categoria = df_20['SUB_CATEGORIA'].unique()


# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_20[df_20['SUB_CATEGORIA'] == sub_categoria]
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()